In [79]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


In [2]:
# !pip install matplotlib

In [80]:
import string
import re
from string import digits
import numpy as np
from sklearn.model_selection import train_test_split
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers import Input, Embedding, LSTM, Dense, dot, concatenate, Activation
import tensorflow as tf
from tensorflow.keras.models import Model

In [81]:
from tensorflow.keras.layers import (Dense,Flatten,SimpleRNN,InputLayer,Conv1D,Bidirectional,GRU,LSTM,BatchNormalization,Dropout,Input, Embedding,TextVectorization)


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!unzip /content/drive/MyDrive/hindi_english_parallel.csv.zip -d "/content/language/"

Archive:  /content/drive/MyDrive/hindi_english_parallel.csv.zip
  inflating: /content/language/hindi_english_parallel.csv  


#### DATA_PREPROCESING

In [82]:
VOCAB_SIZE=20000
ENGLISH_SEQUENCE_LENGTH=32
HINDI_SEQUENCE_LENGTH=32
EMBEDDING_DIM=300
BATCH_SIZE=64


In [83]:
import pandas as pd
lines=pd.read_csv("hindi_english_parallel.csv")
lines.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [84]:
lines.iloc[120086]

hindi      सिर्फ प्रतीक (_ I) 
english           _ Icons Only
Name: 120086, dtype: object

In [85]:
pd.isnull(lines).sum()
lines.dropna(subset=['hindi', 'english'], inplace=True)
pd.isnull(lines).sum()
# len(lines)
lines=lines.sample(n=20000,random_state=42)
print(lines.shape)


(20000, 2)


In [86]:
# Lowercase all characters
lines.english=lines.english.map(lambda x : x.lower())
lines.hindi=lines.hindi.apply(lambda x: x.lower())

# Remove quotes
lines.english=lines.english.apply(lambda x: re.sub("'", '', x))
lines.hindi=lines.hindi.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters

# Remove all the special characters
lines.english=lines.english.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hindi=lines.hindi.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))



# Remove extra spaces
lines.english=lines.english.apply(lambda x: x.strip())
lines.hindi=lines.hindi.apply(lambda x: x.strip())
lines.english=lines.english.apply(lambda x: re.sub(" +", " ", x))
lines.hindi=lines.hindi.apply(lambda x: re.sub(" +", " ", x))

# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.english=lines.english.apply(lambda x: x.translate(remove_digits))
lines.hindi = lines.hindi.apply(lambda x: re.sub("[२३०८१५७९४६0-9]", "", x))



In [87]:
def retain_hindi_alphabet(text):
    return re.sub(r'[^\u0900-\u097F\s]', '', text)
def remove_words_with_numbers(text):
    return re.sub(r'\b[^a-zA-Z\s]+\b', '', text)
def remove_specific_chars(text):
    return re.sub(r'[ं]', '', text)

# Apply the function to the 'hindi' column
lines.hindi = lines.hindi.apply(retain_hindi_alphabet)
lines.english = lines.english.apply(remove_words_with_numbers)
#Apply the function to the 'hindi' column
lines['hindi'] = lines['hindi'].apply(remove_specific_chars)

In [88]:
lines['length_eng_sentence']=lines['english'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi'].apply(lambda x:len(x.split(" ")))
lines=lines[lines['length_eng_sentence']<=ENGLISH_SEQUENCE_LENGTH]
lines=lines[lines['length_hin_sentence']<=HINDI_SEQUENCE_LENGTH-2]
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])
lines.shape

maximum length of Hindi Sentence  30
maximum length of English Sentence  32


(17589, 4)

In [89]:
# Add start and end tokens to target sequences
lines.hindi = lines.hindi.apply(lambda x : 'starttoken '+ x + ' endtoken')
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
136823,starttoken कृपया एक भिन्न नाम उपयोग करे endtoken,please use a different name,5,6
783298,starttoken तेज endtoken,agile,1,1
1216571,starttoken एक अन्य परस्पर जुड़ा हुआ पहलू नवान्...,another inter linked aspect is that of innovat...,10,10
166305,starttoken endtoken,attach the information to bug,6,1
1166752,starttoken कुछ राज्यो मे एक ऊपरी सदन है जिसे र...,certain states have an upper house also called...,11,14


In [ ]:
output=lines.hindi.copy()
# output.head()
output=output.apply(lambda x: re.sub("starttoken", "", x))
output.head()

In [ ]:
english_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=ENGLISH_SEQUENCE_LENGTH
)
hindi_vectorize_layer=TextVectorization(
    standardize='lower_and_strip_punctuation',
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=HINDI_SEQUENCE_LENGTH
)

In [ ]:
english_vectorize_layer.adapt(lines.english)

In [ ]:
hindi_vectorize_layer.adapt(lines.hindi)

In [38]:
lines.hindi=lines.hindi.apply(lambda x: re.sub("endtoken", "", x))
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
892256,starttoken समझ,amount to,2,1
120086,starttoken सिर्फ प्रतीक,icons only,2,3
1184566,starttoken कूपन विनिर्माताओ द्वारा सीधे डाक द्...,coupons may be issued by the manufacturers eit...,15,15
1127237,starttoken आवर्धनः आईपीस की फोकल लम्बाई को ऑब्...,magnification the ratio of the focal length of...,29,30
942384,starttoken फुफ्फुसपाक फुफ्फुसशोथ फेफड़ो का प्र...,pneumonia,1,6


In [39]:
lines.head()

,hindi,english,length_eng_sentence,length_hin_sentence
892256,starttoken समझ,amount to,2,1
120086,starttoken सिर्फ प्रतीक,icons only,2,3
1184566,starttoken कूपन विनिर्माताओ द्वारा सीधे डाक द्...,coupons may be issued by the manufacturers eit...,15,15
1127237,starttoken आवर्धनः आईपीस की फोकल लम्बाई को ऑब्...,magnification the ratio of the focal length of...,29,30
942384,starttoken फुफ्फुसपाक फुफ्फुसशोथ फेफड़ो का प्र...,pneumonia,1,6


In [40]:
def vectorizer(inputs,out):
  return {'input_1':english_vectorize_layer(inputs['english']),
          'input_2':hindi_vectorize_layer(inputs['hindi'])},hindi_vectorize_layer(out)

In [41]:
dataset,output_sol=vectorizer(lines,output)

In [42]:
# output_sol=output_sol.tolist()
print(len(dataset['input_1']), len(output_sol),type(output_sol))

17589 17589 <class 'tensorflow.python.framework.ops.EagerTensor'>


In [43]:
dataset['input_2'].shape

TensorShape([17589, 32])

In [44]:
def data_generator(train, output, batch_size=128):
    while True:
        num_samples = len(output)
        for start_idx in range(0, num_samples, batch_size):
            end_idx = start_idx + batch_size
            batch_input_1 = train['input_1'][start_idx:end_idx]
            batch_input_2 = train['input_2'][start_idx:end_idx]
            batch_output = output[start_idx:end_idx]
            yield (batch_input_1, batch_input_2), batch_output

In [45]:
def valid_generator(train, output):
  batch_input_1 = train['input_1'][0:]
  batch_input_2 = train['input_2'][0:]
  batch_output = output[0:]
  return (batch_input_1, batch_input_2), batch_output

In [46]:
type(dataset['input_1'])

tensorflow.python.framework.ops.EagerTensor

In [47]:
train_size = int(0.30 * len(dataset['input_1']))
train_input_dict = {
    'input_1': dataset['input_1'][:train_size],
    'input_2': dataset['input_2'][:train_size]
}
train_output_list = output_sol[:train_size]

validation_input_dict = {
    'input_1': dataset['input_1'][train_size:],
    'input_2': dataset['input_2'][train_size:]
}
validation_output_list = output_sol[train_size:]

In [48]:
print(len(train_input_dict['input_1']), len(train_output_list),len(validation_output_list))
# NUM_BATCHES=int(lines.shape[0]/BATCH_SIZE)

5276 5276 12313


In [61]:
len_validation=len(validation_output_list)

In [62]:
len_train=len(train_input_dict['input_1'])

In [63]:
NUM_UNITS=256

In [64]:
### ENCODER
input = Input(shape=(ENGLISH_SEQUENCE_LENGTH,), dtype="int64", name="input_1")
x=Embedding(VOCAB_SIZE, EMBEDDING_DIM, )(input)
encoded_input=GRU(NUM_UNITS)(x)

### DECODER
shifted_target=Input(shape=(HINDI_SEQUENCE_LENGTH,), dtype="int64", name="input_2")
x=Embedding(VOCAB_SIZE,EMBEDDING_DIM,)(shifted_target)
x = GRU(NUM_UNITS, return_sequences=True)(x, initial_state=encoded_input)

### OUTPUT
x = Dropout(0.5)(x)
target=Dense(VOCAB_SIZE,activation="softmax")(x)
seq2seq_gru=Model([input,shifted_target],target)
seq2seq_gru.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 32, 300)              6000000   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 32, 300)              6000000   ['input_2[0][0]']             
                                                                                              

In [42]:
plot_model(seq2seq_gru,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [65]:
class BLEU(tf.keras.metrics.Metric):
    def __init__(self,name='bleu_score'):
        super(BLEU,self).__init__()
        self.bleu_score=0

    def update_state(self,y_true,y_pred,sample_weight=None):
      y_pred=tf.argmax(y_pred,-1)
      self.bleu_score=0
      for i,j in zip(y_pred,y_true):
        tf.autograph.experimental.set_loop_options()

        total_words=tf.math.count_nonzero(i)
        total_matches=0
        for word in i:
          if word==0:
            break
          for q in range(len(j)):
            if j[q]==0:
              break
            if word==j[q]:
              total_matches+=1
              j=tf.boolean_mask(j,[False if y==q else True for y in range(len(j))])
              break

        self.bleu_score+=total_matches/total_words

    def result(self):
        return self.bleu_score/BATCH_SIZE

In [66]:
seq2seq_gru.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(5e-4))
    # metrics=[BLEU()], 
    # run_eagerly=True)

In [67]:
checkpoint_filepath = 'nmt_lstm.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True,)

In [68]:

train_generator = data_generator(train_input_dict, train_output_list, BATCH_SIZE)
validation_generator = data_generator(validation_input_dict, validation_output_list,BATCH_SIZE)

validation_data=validation_generator,
history= seq2seq_gru.fit(
   train_generator,
   steps_per_epoch = len_train//BATCH_SIZE,
   epochs=12,
    callbacks=[model_checkpoint_callback],
    verbose=1
   )


Epoch 1/12
82/82 [==============================] - ETA: 0s - loss: 4.7221

/home/admin1/venv/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


82/82 [==============================] - 53s 619ms/step - loss: 4.7221
Epoch 2/12
82/82 [==============================] - 51s 619ms/step - loss: 2.5100
Epoch 3/12
82/82 [==============================] - 50s 615ms/step - loss: 2.2467
Epoch 4/12
82/82 [==============================] - 51s 616ms/step - loss: 1.9812
Epoch 8/12
82/82 [==============================] - 51s 618ms/step - loss: 1.9551
Epoch 9/12
82/82 [==============================] - 51s 617ms/step - loss: 1.9264
Epoch 10/12
82/82 [==============================] - 51s 614ms/step - loss: 1.9025
Epoch 11/12
82/82 [==============================] - 51s 622ms/step - loss: 1.8800
Epoch 12/12
82/82 [==============================] - 51s 618ms/step - loss: 1.8551


In [69]:
index_to_word={x:y for x, y in zip(range(len(hindi_vectorize_layer.get_vocabulary())),
                                   hindi_vectorize_layer.get_vocabulary())}

In [76]:
seq2seq_gru = tf.keras.models.load_model('nmt_weights.h5')

ValueError: No model config found in the file at <tensorflow.python.platform.gfile.GFile object at 0x7f278d091970>.

In [70]:
def translator(english_sentence):
  tokenized_english_sentence=english_vectorize_layer([english_sentence])
  shifted_target='starttoken'

  for i in range(HINDI_SEQUENCE_LENGTH):
    tokenized_shifted_target=hindi_vectorize_layer([shifted_target])
    output=seq2seq_gru.predict([tokenized_english_sentence,tokenized_shifted_target])
    french_word_index=tf.argmax(output,axis=-1)[0][i].numpy()
    current_word=index_to_word[french_word_index]
    if current_word=='endtoken':
      break
    shifted_target+=' '+current_word
  return shifted_target[11:]

In [75]:
translator('hello who are you')

1/1 [==============================] - 0s 17ms/step


'[UNK]'

In [ ]:
word_to_index={y:x for x, y in zip(range(len(hindi_vectorize_layer.get_vocabulary())),
                                   hindi_vectorize_layer.get_vocabulary())}

In [78]:
seq2seq_gru.evaluate(train_generator)

    187/Unknown - 42s 224ms/step - loss: 1.8050

KeyboardInterrupt: 